In [2]:
import sys
import torch
from pathlib import Path
import os
from PIL import Image
import shutil
import cv2
from models.common import DetectMultiBackend

ModuleNotFoundError: No module named 'models'

In [4]:
# Caminhos do seu dataset
train_dir = r'C:\Users\MarcosHenriqueSaito\Documents\Oficina\MarcosYoloV\Treino'
test_dir = r'C:\Users\MarcosHenriqueSaito\Documents\Oficina\MarcosYoloV\Teste'
train_label_dir = r'C:\Users\MarcosHenriqueSaito\Documents\Oficina\MarcosYoloV\Labels\Treino'
test_label_dir = r'C:\Users\MarcosHenriqueSaito\Documents\Oficina\MarcosYoloV\Labels\Teste'

# Criar diretórios para as anotações se não existirem
os.makedirs(train_label_dir, exist_ok=True)
os.makedirs(test_label_dir, exist_ok=True)

# Função para gerar arquivo de anotação YOLO
def generate_annotation(image_path, label_dir):
    # Supondo que o rosto esteja sempre na mesma posição ou conhecida
    # Exemplo de caixa delimitadora: [xmin, ymin, xmax, ymax] em pixels
    # Para um exemplo simples, vamos colocar uma caixa fixa (ajuste conforme necessário)
    xmin, ymin, xmax, ymax = 100, 100, 400, 400  # Exemplo de caixa delimitadora

    # Carregar a imagem
    img = Image.open(image_path)
    width, height = img.size

    # Normalizar a caixa delimitadora
    x_center = (xmin + xmax) / 2 / width
    y_center = (ymin + ymax) / 2 / height
    box_width = (xmax - xmin) / width
    box_height = (ymax - ymin) / height

    # Gerar o arquivo de anotação
    annotation = f"0 {x_center} {y_center} {box_width} {box_height}\n"  # '0' é a classe (rosto)

    # Nome da imagem sem a extensão
    image_name = os.path.splitext(os.path.basename(image_path))[0]
    
    # Caminho para salvar o arquivo de anotação
    label_path = os.path.join(label_dir, f"{image_name}.txt")

    # Salvar a anotação
    with open(label_path, 'w') as f:
        f.write(annotation)

# Função para percorrer diretórios e gerar anotações para as imagens
def process_images(image_dir, label_dir):
    for image_name in os.listdir(image_dir):
        if image_name.endswith(".jpg") or image_name.endswith(".jpeg") or image_name.endswith(".png"):
            image_path = os.path.join(image_dir, image_name)
            generate_annotation(image_path, label_dir)

# Processar imagens de treino
process_images(train_dir, train_label_dir)

# Processar imagens de teste
process_images(test_dir, test_label_dir)

print("Arquivos de anotação gerados com sucesso!")


Arquivos de anotação gerados com sucesso!


In [ ]:
# Adicionar o caminho para o repositório YOLOv5
sys.path.append(str(Path('D:/UTF/7 Periodo/Oficina/projeto/yolov5')))  # ajuste conforme o caminho exato

In [3]:
model_path = 'D:/UTF/7 Periodo/Oficina/projeto/yolov5/runs/train/exp12/weights/best.pt'
model = DetectMultiBackend(model_path)
model.eval()


Fusing layers... 
YOLOv5n summary: 157 layers, 1760518 parameters, 0 gradients, 4.1 GFLOPs


DetectMultiBackend(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
        (act): SiLU(inplace=True)
      )
      (2): C3(
        (cv1): Conv(
          (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (cv3): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
          (act): SiLU(inplace=True)
        )
        (m): Sequential(
          (0): Bottleneck(
            (cv1): Conv(
              (conv): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
              (act): SiLU(inplace=True)
            )
            (cv2): Conv(

In [4]:
# Inicializar a câmera
cap = cv2.VideoCapture(0)  # 0 geralmente é o índice da webcam padrão
if not cap.isOpened():
    print("Erro ao abrir a câmera")
    exit()


In [6]:
while True:
    ret, frame = cap.read()
    if not ret:
        print("Erro ao capturar o quadro")
        break

    # Pré-processamento para YOLOv5
    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # YOLOv5 espera imagens em RGB
    img = torch.from_numpy(img).permute(2, 0, 1).unsqueeze(0).float()  # Converte o frame para um tensor PyTorch

    # Fazer previsão
    predictions = model(img)  # Realiza a previsão

    # Processar as previsões para desenhar as caixas delimitadoras
    for det in predictions[0]:  # Acessa as detecções no primeiro índice
        # Extraímos os valores da detecção
        x1, y1, x2, y2, conf, cls = det[0].item(), det[1].item(), det[2].item(), det[3].item(), det[4].item(), det[5].item()

        # Desenha a caixa delimitadora
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        label = f"Class {int(cls)}: {conf:.2f}"
        cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Exibir o frame com as caixas delimitadoras
    cv2.imshow('YOLOv5 Nano - Face Recognition', frame)

    # Pressione 'q' para sair
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


RuntimeError: a Tensor with 6 elements cannot be converted to Scalar

In [ ]:
# Liberar a câmera e fechar todas as janelas
cap.release()
cv2.destroyAllWindows()